In [1]:
import numpy as np

##### 1. [2 marks] Implement the weighted h-categorizer semantics, using a parameter $\epsilon > 0$ to control the convergence.

In [2]:
# A : liste des arguments
A = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']

# R : liste de tuples (attaquant, attaqué)
R = [('A', 'B'), ('A', 'C'), ('B', 'D'), ('C', 'D'), ('D', 'E'), ('F', 'E'), ('G', 'F'), ('H', 'G')]

# w : dictionnaire {argument: poids initial}
w = {'A': 0.9, 'B': 0.8, 'C': 0.7, 'D': 0.6, 'E': 0.5, 'F': 0.4, 'G': 0.3, 'H': 0.2}

epsilon = 0.01  # seuil de convergence*
max_iter = 100  # nombre maximum d'itérations

In [3]:
def build_att(A, R):
    att_list = {a: [] for a in A}
    for att, target in R:
        if target in att_list:
            att_list[target].append(att)
        else:
            att_list[target] = [att]
    return att_list

In [4]:
def h_categorizer(A, R, w, max_iter, epsi=1e-4):

    #on construit dans un premier temps le dictionnaire des attaques
    # on ajoute pour chaque argument a une liste des arguments qui l'attaquent
    attaquand = build_att(A, R)

    # on initialise les poids courants avec les poids initiaux
    hc = {a: w[a] for a in A}

    for iteration in range(max_iter):
        # On crée un nouveau dictionnaire pour stocker les valeurs calculées à cette itération
        new_hc = {}

        for a in A:
            liste_attaquants = attaquand[a]
            # On calcule la somme des valeurs hc actuelles de tous ces attaquants
            somme_attaquants = 0
            for b in liste_attaquants:
                somme_attaquants += hc[b]

            # update de la valeur de 'a' avec la formule : HC_{k+1}(a) = w(a) / (1 + somme des attaquants) puis on save
            newVal = w[a] / (1 + somme_attaquants)
            new_hc[a] = newVal

        # On vérifie la convergence en comparant les nouvelles valeurs avec les anciennes
        diff = max(abs(new_hc[a] - hc[a]) for a in A)
        hc = new_hc

        if diff < epsi:
            break

    return hc

In [5]:
h_categorizer(A,R,w,max_iter,epsilon)

{'A': 0.9,
 'B': 0.4210526315789474,
 'C': 0.3684210526315789,
 'D': 0.3352941176470588,
 'E': 0.30206112295664533,
 'F': 0.32,
 'G': 0.25,
 'H': 0.2}

##### 2. [3 marks] Note that by fixing an arbitrary order on $A$, we can convert $w$ into a vector in $[0,1]^A$. For example, if $w(a)=0.2$ and $w(b)=0.5$, we obtain the vector $[0.2,0.5]$. Thus, for a given argumentation framework $(A, R)$ without weights, the semantics $\Sigma_{HC}$ converts $w$ into a vector $x \in [0,1]^A$. Implement a program that:
- a. Take as input an argumentation framework $(A,R)$
- b. Sample 100,000 random vectors $w_i \in [0,1]^A$ and compute the corresponding vectors $x_i \in [0,1]^A$.
- c. Construct the convex hull of the points $x_i$. This hull is an approximation of the space of acceptability degree of $(A,R)$ for the h-categorizer semantics.

In [6]:
def dict_to_vector(A, d):
    return np.array([d[a] for a in A], dtype=float)

In [7]:
dict_to_vector(A,w)

array([0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2])

###### a. Take as input an argumentation framework $(A,R)$
###### b. Sample 100,000 random vectors $w_i \in [0,1]^A$ and compute the corresponding vectors $x_i \in [0,1]^A$.

In [8]:
def sample_and_compute_X(A, R, epsilon,max_iter=1000, n_samples=10000, seed=42):

    #générateur de nombres aléatoires pour les vecteurs
    rng = np.random.default_rng(seed)

    # on initialise la matrice des résultats
    X = np.zeros((n_samples, len(A)), dtype=float)

    for i in range(n_samples):

        # 1. Génère un vecteur aléatoire de poids w
        w_rng_values = rng.random(len(A))
        w = dict(zip(A, w_rng_values))

        # 2. Calcule le vecteur x correspondant (Σ_HC)
        HC = h_categorizer(A, R, w, max_iter,epsilon)

        # 3. Enregistre ce vecteur dans la matrice X
        X[i, :] = dict_to_vector(A, HC)

    return X





In [9]:
newA = ['A', 'B', 'C', 'D']
X_results = sample_and_compute_X(newA,R,epsilon, max_iter=1000)
X_results

array([[0.77395605, 0.24740097, 0.4840018 , 0.40277632],
       [0.09417735, 0.8916492 , 0.69562736, 0.3038192 ],
       [0.12811363, 0.3992381 , 0.32868854, 0.5363451 ],
       ...,
       [0.19022548, 0.58581537, 0.76026674, 0.13929154],
       [0.42067413, 0.68149865, 0.1022618 , 0.49309022],
       [0.9592852 , 0.40805945, 0.17169826, 0.41372238]])

c. Construct the convex hull of the points $x_i$. This hull is an approximation of the space of acceptability degree of $(A,R)$ for the h-categorizer semantics.

In [10]:
from scipy.spatial import ConvexHull

envConvex = ConvexHull(X_results)